In [2]:
from subgrounds import Subgrounds
from subgrounds.pagination import ShallowStrategy
import pandas as pd


In [3]:
endpoint = 'https://api.thegraph.com/subgraphs/name/maverickprotocol/maverick-mainnet-app'
sg = Subgrounds()
endpoint = endpoint
subgraph = sg.load_subgraph(endpoint)

CONFIG_COMMON_SWAPS_TOKENS = {
    "ethereum": {
        # [GHO]
        'search_tokens': ["0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f", ],
    }
}


In [4]:
# Build the query
chain = 'ethereum'
config = CONFIG_COMMON_SWAPS_TOKENS[chain]
last_materialized_timestamp = 1693362587
last_materialized_block = 18024449


tokens_where = []
tokens_where.append({
    'pool_': {'tokenA_in': config['search_tokens']},
    'timestamp_gt': last_materialized_timestamp
})
tokens_where.append({
    'pool_': {'tokenB_in': config['search_tokens']},
    'timestamp_gt': last_materialized_timestamp
})

swaps_data = pd.DataFrame()
for w in tokens_where:
    swaps = subgraph.Query.swaps(
        where=w,
        first=1000000
    )

    field_paths = [
        swaps.id,
        swaps.timestamp,
        swaps.pool.id,
        swaps.pool.fee,
        swaps.pool.tokenA.id,
        swaps.pool.tokenB.id,
        swaps.pool.tokenA.symbol,
        swaps.pool.tokenB.symbol,
        # swaps.from,
        swaps.sender,
        swaps.recipient,
        swaps.tokenAIn,
        swaps.amountIn,
        swaps.amountInUSD,
        swaps.amountOut,
        swaps.amountOutUSD,
    ]

    part = sg.query_df(field_paths, pagination_strategy=ShallowStrategy)
    swaps_data = pd.concat([swaps_data, part])

# swaps_data
swaps_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   swaps_id                  11 non-null     object 
 1   swaps_timestamp           11 non-null     int64  
 2   swaps_pool_id             11 non-null     object 
 3   swaps_pool_fee            11 non-null     float64
 4   swaps_pool_tokenA_id      11 non-null     object 
 5   swaps_pool_tokenB_id      11 non-null     object 
 6   swaps_pool_tokenA_symbol  11 non-null     object 
 7   swaps_pool_tokenB_symbol  11 non-null     object 
 8   swaps_sender              11 non-null     object 
 9   swaps_recipient           11 non-null     object 
 10  swaps_tokenAIn            11 non-null     bool   
 11  swaps_amountIn            11 non-null     float64
 12  swaps_amountInUSD         11 non-null     float64
 13  swaps_amountOut           11 non-null     float64
 14  swaps_amount

In [5]:
cp = swaps_data.copy()


In [6]:
swaps_data = cp.copy()
swaps_data.rename(columns={
    'swaps_id': 'tx_hash',
    'swaps_timestamp': 'block_unixtimestamp',
    'swaps_pool_id': 'pool_address',
    'swaps_pool_fee': 'fee_tier',
    'swaps_pool_tokenA_id': 'tokenA_address',
    'swaps_pool_tokenB_id': 'tokenB_address',
    'swaps_pool_tokenA_symbol': 'tokenA_symbol',
    'swaps_pool_tokenB_symbol': 'tokenB_symbol',
    'swaps_sender': 'sender_address',
    'swaps_recipient': 'recipient_address',
    'swaps_tokenAIn': 'tokenA_in',
    'swaps_amountIn': 'amount_in',
    'swaps_amountInUSD': 'amount_in_usd',
    'swaps_amountOut': 'amount_out',
    'swaps_amountOutUSD': 'amount_out_usd',
}, inplace=True)

swaps_data['block_timestamp'] = pd.to_datetime(
    swaps_data['block_unixtimestamp'], unit='s')
swaps_data['chain'] = chain
swaps_data['platform'] = 'maverick'
swaps_data['is_aggregator'] = False
# swaps_data['recipient_address'] = swaps_data.sender_address
# swaps_data['ca'] = None
swaps_data['event_index'] = None
swaps_data['caller_address'] = None
swaps_data['pool_id'] = swaps_data.pool_address
swaps_data['fee_tier'] = None
swaps_data['block_number'] = None


def assign_directions(row):
    if row.tokenA_in:
        row['token_in_address'] = row.tokenA_address
        row['token_in_symbol'] = row.tokenA_symbol
        row['token_in_amount'] = row.amount_in
        row['token_in_amount_usd'] = row.amount_in_usd
        row['token_out_address'] = row.tokenB_address
        row['token_out_symbol'] = row.tokenB_symbol
        row['token_out_amount'] = row.amount_out
        row['token_out_amount_usd'] = row.amount_out_usd
    else:
        row['token_in_address'] = row.tokenB_address
        row['token_in_symbol'] = row.tokenB_symbol
        row['token_in_amount'] = row.amount_out
        row['token_in_amount_usd'] = row.amount_out_usd
        row['token_out_address'] = row.tokenA_address
        row['token_out_symbol'] = row.tokenA_symbol
        row['token_out_amount'] = row.amount_in
        row['token_out_amount_usd'] = row.amount_in_usd
    return row


swaps_data = swaps_data.apply(assign_directions, axis=1)

# swaps_data['token_in_symbol'] = None
# swaps_data['token_out_symbol'] = None

# # swaps_data['event_index'] = swaps_data.event_index.astype(int)
# # swaps_data['event_index'] = swaps_data.apply(
# #     lambda row: row['swaps_id'].replace(row['tx_hash'], ''), axis=1)


swaps_data['token_in_price_usd'] = swaps_data.token_in_amount_usd / \
    swaps_data.token_in_amount
swaps_data['token_out_price_usd'] = swaps_data.token_out_amount_usd / \
    swaps_data.token_out_amount


swaps_data = swaps_data[[
    "tx_hash",
    "event_index",
    "block_timestamp",
    "block_unixtimestamp",
    "block_number",
    "chain",
    "platform",
    "pool_address",
    "pool_id",
    "fee_tier",
    "is_aggregator",
    "caller_address",
    "sender_address",
    "recipient_address",
    "token_in_address",
    "token_out_address",
    "token_in_symbol",
    "token_out_symbol",
    "token_in_amount",
    "token_in_amount_usd",
    "token_in_price_usd",
    "token_out_amount",
    "token_out_amount_usd",
    "token_out_price_usd"
]]

# swaps_data.sort_values(by=['block_number', 'event_index'], inplace=True)
# last_block = swaps_data.block_number.max()
# last_timestamp = swaps_data.block_unixtimestamp.max()


In [7]:
swaps_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   tx_hash               11 non-null     object        
 1   event_index           0 non-null      object        
 2   block_timestamp       11 non-null     datetime64[ns]
 3   block_unixtimestamp   11 non-null     int64         
 4   block_number          0 non-null      object        
 5   chain                 11 non-null     object        
 6   platform              11 non-null     object        
 7   pool_address          11 non-null     object        
 8   pool_id               11 non-null     object        
 9   fee_tier              0 non-null      object        
 10  is_aggregator         11 non-null     bool          
 11  caller_address        0 non-null      object        
 12  sender_address        11 non-null     object        
 13  recipient_address     

In [8]:
swaps_data


,tx_hash,event_index,block_timestamp,block_unixtimestamp,block_number,chain,platform,pool_address,pool_id,fee_tier,...,token_in_address,token_out_address,token_in_symbol,token_out_symbol,token_in_amount,token_in_amount_usd,token_in_price_usd,token_out_amount,token_out_amount_usd,token_out_price_usd
0,0x19f781349706ab0f85e76f751b96ebf69f988d50752b...,None,2023-08-30 20:31:59,1693427519,None,ethereum,maverick,0x2e9b3deca040abe4cb11b8ba4135da6f379a44b0,0x2e9b3deca040abe4cb11b8ba4135da6f379a44b0,None,...,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,0x5f98805a4e8be255a32880fdec7f6728c6568ba0,GHO,LUSD,64254.369562,0.0,0.0,63068.542251,6.322859e+04,1.002538e+00
1,0x261584c412b2ba4f9081ff6fec2038129a6b006c259c...,None,2023-09-22 07:42:11,1695368531,None,ethereum,maverick,0x2e9b3deca040abe4cb11b8ba4135da6f379a44b0,0x2e9b3deca040abe4cb11b8ba4135da6f379a44b0,None,...,0x5f98805a4e8be255a32880fdec7f6728c6568ba0,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,LUSD,GHO,14454.706877,0.0,0.0,14239.991904,3.606295e+10,2.532512e+06
2,0x2910ab8cd1ee23d8ab59ea198dbc78f0c381e7786108...,None,2023-09-21 11:41:23,1695296483,None,ethereum,maverick,0x2e9b3deca040abe4cb11b8ba4135da6f379a44b0,0x2e9b3deca040abe4cb11b8ba4135da6f379a44b0,None,...,0x5f98805a4e8be255a32880fdec7f6728c6568ba0,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,LUSD,GHO,76101.145943,0.0,0.0,74822.411826,1.919607e+11,2.565551e+06
3,0x4dfdcd916a555288348493af0d858149fdfbcb4b90ff...,None,2023-09-20 02:40:23,1695177623,None,ethereum,maverick,0x2e9b3deca040abe4cb11b8ba4135da6f379a44b0,0x2e9b3deca040abe4cb11b8ba4135da6f379a44b0,None,...,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,0x5f98805a4e8be255a32880fdec7f6728c6568ba0,GHO,LUSD,21969.492486,0.0,0.0,21575.140421,2.173625e+04,1.007467e+00
4,0x5ae05702e67c54339763990af1cf6f22618866d24f58...,None,2023-09-21 14:32:23,1695306743,None,ethereum,maverick,0x2e9b3deca040abe4cb11b8ba4135da6f379a44b0,0x2e9b3deca040abe4cb11b8ba4135da6f379a44b0,None,...,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,0x5f98805a4e8be255a32880fdec7f6728c6568ba0,GHO,LUSD,29364.270122,0.0,0.0,28867.672507,7.361094e+10,2.549944e+06
5,0x5f63e516a392b405000ee9da06af797f4c883840acbb...,None,2023-09-22 12:44:59,1695386699,None,ethereum,maverick,0x2e9b3deca040abe4cb11b8ba4135da6f379a44b0,0x2e9b3deca040abe4cb11b8ba4135da6f379a44b0,None,...,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,0x5f98805a4e8be255a32880fdec7f6728c6568ba0,GHO,LUSD,11551.959091,0.0,0.0,11381.430887,2.922703e+10,2.567958e+06
6,0x71907f58aacc52c517c6b24e2fd90f5406717f73c06c...,None,2023-08-30 23:18:11,1693437491,None,ethereum,maverick,0x2e9b3deca040abe4cb11b8ba4135da6f379a44b0,0x2e9b3deca040abe4cb11b8ba4135da6f379a44b0,None,...,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,0x5f98805a4e8be255a32880fdec7f6728c6568ba0,GHO,LUSD,2446.167686,0.0,0.0,2399.459442,2.396937e+03,9.989486e-01
7,0x81631b2be933bcc74ea16cab4e6be22aa7b04f5edce4...,None,2023-09-22 15:47:59,1695397679,None,ethereum,maverick,0x2e9b3deca040abe4cb11b8ba4135da6f379a44b0,0x2e9b3deca040abe4cb11b8ba4135da6f379a44b0,None,...,0x5f98805a4e8be255a32880fdec7f6728c6568ba0,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,LUSD,GHO,9180.256335,0.0,0.0,9049.942941,2.322549e+10,2.566369e+06
8,0xa774c52d3373c654f3a1b31c2a164c778b3939e843df...,None,2023-08-30 04:09:35,1693368575,None,ethereum,maverick,0x2e9b3deca040abe4cb11b8ba4135da6f379a44b0,0x2e9b3deca040abe4cb11b8ba4135da6f379a44b0,None,...,0x5f98805a4e8be255a32880fdec7f6728c6568ba0,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,LUSD,GHO,29134.511681,0.0,0.0,28622.026074,2.871538e+04,1.003262e+00
9,0xda76430dcabda2c06d2b81977a6dcea3e1b2d3523f41...,None,2023-09-22 07:57:23,1695369443,None,ethereum,maverick,0x2e9b3deca040abe4cb11b8ba4135da6f379a44b0,0x2e9b3deca040abe4cb11b8ba4135da6f379a44b0,None,...,0x5f98805a4e8be255a32880fdec7f6728c6568ba0,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,LUSD,GHO,31638.915131,0.0,0.0,31184.000941,7.897385e+10,2.532512e+06


In [9]:
l = int(swaps_data.block_number.max())


ValueError: cannot convert float NaN to integer